In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import halfnorm
from scipy.stats import expon
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib

from sfma.api import SFMAModel

In [ ]:
class Simulator:
    def __init__(self, nu: int, gamma: int, sigma_min: float, sigma_max: float,
                 x: callable, func: callable, ineff_dist: str = 'half-normal'):
        """
        Simulation class for stochastic frontier meta-analysis.
        
        nu
            The scale of the inefficiency term
        gamma
            The variance of the random effect term
        sigma_min, sigma_max
            The study-specific errors, max and minimum. They will be drawn from a uniform distribution.
        x
            A callable function to generate a realization from a random variable x (is the covariate used
            to construct the frontier). Needs to have an argument size.
        func
            A function of x that defines the frontier
        ineff_dist
            Inefficiency distribution
        """
        self.nu = nu
        self.gamma = gamma
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max
        self.x = x
        self.func = func
        
        if ineff_dist == 'half-normal':
            self.rvs = halfnorm.rvs
        elif ineff_dist == 'exponential':
            self.rvs = expon.rvs
        else:
            raise RuntimeError("Inefficiency distribution must be half-normal or exponential")
    
    def simulate(self, n: int = 1, **kwargs):
        np.random.seed(365)
        sigma = stats.uniform.rvs(loc=self.sigma_min, scale=self.sigma_max, size=n)
        epsilon = stats.norm.rvs(loc=0, scale=sigma, size=n)
        
        us = stats.norm.rvs(loc=0, scale=self.gamma, size=n)
        vs = self.rvs(scale=self.nu, size=n)
        
        xs = self.x(size=n, **kwargs)
        front = self.func(xs)
        observed = front + us - vs + epsilon
        return us, vs, epsilon, sigma, xs, front, observed

In [ ]:
np.random.seed(150)

## Figure 1

Data generating process

In [ ]:
s = Simulator(nu=1, gamma=0.25, sigma_min=0, sigma_max=0.75, 
              x=lambda size: stats.uniform.rvs(size=size, loc=0.5), func=lambda x: np.log(x) + 10)
us, vs, epsilon, sigma, xs, front, observed = s.simulate(n = 30)

x_front = np.linspace(xs.min(), xs.max())
y_front = s.func(np.linspace(xs.min(), xs.max()))

ymin = observed.min() - 0.5
ymax = observed.max() + 0.5

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(15, 10))

gs = fig.add_gridspec(3, 2)
ax0 = fig.add_subplot(gs[0, :-1])
ax1 = fig.add_subplot(gs[0, -1])
ax2 = fig.add_subplot(gs[1, :-1])
ax3 = fig.add_subplot(gs[1, -1])
ax4 = fig.add_subplot(gs[2, :])

ax0.scatter(xs, front, color='grey')
ax0.plot(x_front, y_front, color='black', label='frontier')
ax0.set_ylim([ymin, ymax])
ax0.legend(loc='upper left')
ax0.set_ylabel("Output Value")

ax1.plot(x_front, y_front, color='black')
ax1.vlines(xs, ymin=front, ymax=front+us, color='blue', label='random effect')
pos_change = us > 0
ax1.scatter(xs[pos_change], (front+us)[pos_change], color='blue', marker='^')
ax1.scatter(xs[~pos_change], (front+us)[~pos_change], color='blue', marker='v')
ax1.set_ylim([ymin, ymax])
ax1.legend(loc='upper left')

ax2.plot(x_front, y_front, color='black')
ax2.vlines(xs, ymin=front, ymax=front+us, color='blue', linestyle=':')
ax2.vlines(xs, ymin=front+us-vs, ymax=front+us, color='green', label='inefficiency')
ax2.scatter(xs, front+us-vs, color='green', marker='v')
ax2.set_ylim([ymin, ymax])
ax2.legend(loc='upper left')
ax2.set_ylabel("Output Value")

ax3.plot(x_front, y_front, color='black')
ax3.vlines(xs, ymin=front, ymax=front+us, color='blue', linestyle=':')
ax3.vlines(xs, ymin=front+us-vs, ymax=front+us, color='green', linestyle=':')
ax3.vlines(xs, ymin=front+us-vs, ymax=front+us-vs+epsilon, color='red', label="study error")
pos_change = epsilon > 0
ax3.scatter(xs[pos_change], (front+us-vs+epsilon)[pos_change], color='red', marker='^')
ax3.scatter(xs[~pos_change], (front+us-vs+epsilon)[~pos_change], color='red', marker='v')
ax3.set_ylim([ymin, ymax])
ax3.legend(loc='upper left')

ax4.scatter(xs, front+us-vs+epsilon, color='black', label='observation')
ax4.plot(x_front, y_front, color='black')
ax4.vlines(xs, ymin=front, ymax=front+us, color='blue', linestyle=':')
ax4.vlines(xs, ymin=front+us-vs, ymax=front+us, color='green', linestyle=':')
ax4.vlines(xs, ymin=front+us-vs, ymax=front+us-vs+epsilon, color='red', linestyle=':')
ax4.set_ylim([ymin, ymax])
ax4.legend(loc='upper left')
ax4.set_xlabel("Input Value")
ax4.set_ylabel("Output Value")

plt.savefig("data-generating-process.pdf")

## Figure 2

Single model fit

In [ ]:
s = Simulator(nu=0.4, gamma=0.01, sigma_min=0.1, sigma_max=0.2, ineff_dist='half-normal',
              x=lambda size: stats.uniform.rvs(size=size, loc=0.0, scale=10), func=lambda x: np.log(5 * x + 1.5))
us, vs, epsilon, sigma, xs, front, observed = s.simulate(n=1000)

x_front = np.linspace(xs.min(), xs.max())
y_front = s.func(np.linspace(xs.min(), xs.max()))

In [ ]:
sim = pd.DataFrame({'output': observed, 'input': xs, 'se': sigma})
sim.sort_values('input', inplace=True)

In [ ]:
model = SFMAModel(
    df=sim,
    col_output='output',
    col_se='se',
    col_input='input',
    concave=True,
    include_gamma=True,
    increasing=True,
    pct_trimming=0.1
)
model.fit()

In [ ]:
model.plot_inefficiencies()

In [ ]:
model.plot_frontier(sim, true_frontier=np.array([x_front, y_front]).T)

In [ ]:
model.solver.result['success']

In [ ]:
model.marginal_model.objective(data=model.data, x=increasing_opt)

In [ ]:
model.marginal_model.objective(data=model.data, x=model.solver.x_opt)

In [ ]:
# increasing
increasing_opt = model.solver.x_opt

In [ ]:
model.solver.result

In [ ]:
model.solver.x_opt

In [ ]:
model.marginal_model.objective(data=model.data, x=model.solver.x_opt)

In [ ]:
x_try = model.solver.x_opt
x_try[0:model.marginal_model.femat.shape[1]] -= 1.5
x_try[-1] = 0.11007753575536437

In [ ]:
model.marginal_model.objective(data=model.data, x=x_try)

In [ ]:
model.solver.result

In [ ]:
model.solver.x_opt

Gamma

In [ ]:
0.2**2

This is what we simulated for the scale of half-normal

In [ ]:
0.2

Estimated the variance of the normal, then when we did this transformation, we estimated standard deviation for the half-normal 

In [ ]:
np.sqrt(1 - 2/np.pi)*np.sqrt(2.2239941)

In [ ]:
(0.2/np.sqrt(1 - 2/np.pi))**2

## Run through many versions of simulations and try different model fits

In [ ]:
def sim_fit(nu, gamma, sigma_ran):
    
    s = Simulator(nu=nu, gamma=gamma, sigma_min=sigma_ran[0], sigma_max=sigma_ran[1], ineff_dist='half-normal',
                  x=lambda size: stats.uniform.rvs(size=size, loc=0.0, scale=10), func=lambda x: np.log(5 * x + 1.5))
    us, vs, epsilon, sigma, xs, front, observed = s.simulate(n = 500)

    sim = pd.DataFrame({'output': observed, 'input': xs, 'se': sigma})
    sim.sort_values('input', inplace=True)

    x_front = np.linspace(xs.min(), xs.max())
    y_front = s.func(np.linspace(xs.min(), xs.max()))

    # Look at linear tail versus not on this one
    for concave in [True, False]:
        for increasing in [True, False]:
            for include_gamma in [True, False]:
                name = ""
                if concave:
                    name += "concave "
                if increasing:
                    name += "increasing "
                if include_gamma:
                    name += "gamma "
                if name == "":
                    name = "basic "
                model = SFMAModel(
                    df=sim,
                    col_output='output',
                    col_se='se',
                    col_input='input',
                    concave=concave,
                    include_gamma=include_gamma,
                    increasing=increasing
                )
                error = False
                try:
                    model.fit(options={'solver_options': {}})
                    converged = model.solver.result['success']
                except ValueError as e:
                    if "array must not contain infs or NaNs" in str(e):
                        error = True
                        converged = False
                    else:
                        raise e
                if converged:
                    name += "(+)"
                else:
                    if error:
                        name += "(x)"
                    else:
                        name += "(-)"
                print(name, "\r")
                if not error:
                    preds = model.predict()
                    sim[name] = preds
                    
                    initial_values = model.marginal_model.get_var_init(data=model.data)
                    model.solver.x_opt = initial_values
                    initial_preds = model.predict()
                    
                    sim['init'] = initial_preds
                else:
                    sim[name] = np.nan
    return sim

In [ ]:
this_sim = sim_fit(0.5, 0.1, [0.1, 0.2])

In [ ]:
sims = dict()

j = 0
for NU in [0.01, 0.1, 0.5]:
    print(f"NU {NU}")
    sims[NU] = dict()
    for GAMMA in [0.0, 0.1]:
        print(f"GAMMA {GAMMA}")
        sims[NU][GAMMA] = dict()
        for SIGMA_RAN in [[0.01, 0.02], [0.1, 0.2]]:
            SIGMA = '-'.join([str(x) for x in SIGMA_RAN])
            print(f"SIGMA {SIGMA}")
            print(f"ITER {j}")
            this_sim = sim_fit(NU, GAMMA, SIGMA_RAN)
            sims[NU][GAMMA][SIGMA] = this_sim
            j += 1

In [ ]:
stys = ['solid', 'dashed']
cols = ['red', 'blue']

fig, ax = plt.subplots(12, 2, figsize=(18, 8*12))

j = 0
for NU in [0.01, 0.1, 0.5]:
    for GAMMA in [0.0, 0.1]:
        for SIGMA_RAN in [[0.01, 0.02], [0.1, 0.2]]:
            SIGMA = '-'.join([str(x) for x in SIGMA_RAN])
            this_sim = sims[NU][GAMMA][SIGMA]
            for concave in [True, False]:
                if concave:
                    sty = stys[0]
                else:
                    sty = stys[1]
                for increasing in [True, False]:
                    if increasing:
                        clr = cols[0]
                    else:
                        clr = cols[1]
                    i = 0
                    for include_gamma in [True, False]:
                        name = ""
                        if concave:
                            name += "concave "
                        if increasing:
                            name += "increasing "
                        if include_gamma:
                            name += "gamma "
                        if name == "":
                            name = "basic "
                        if name + "(+)" in this_sim.columns:
                            col = name + "(+)"
                        elif name + "(-)" in this_sim.columns:
                            col = name + "(-)"
                        elif name + "(x)" in this_sim.columns:
                            col = name + "(x)"

                        ax[j,i].plot(x_front, y_front, linestyle='solid', color='black', alpha=0.5)
                        ax[j,i].errorbar(this_sim['input'], this_sim['output'], yerr=sim['se'], linestyle='None', color='black', alpha=0.05)
                        ax[j,i].scatter(this_sim['input'], this_sim['output'], color='black', alpha=0.05)
                        ax[j,i].plot(this_sim['input'], this_sim[col], c=clr, linestyle=sty, label=col)
                        ax[j,i].plot(this_sim['input'], this_sim['init'], c='green', label='init')

                        ax[j,i].legend()
                        title = f"NU: {NU}, GAMMA: {GAMMA}, SIGMA: {SIGMA}"
                        if i == 0:
                            title = "WITH GAMMA -- " + title
                        if i == 1:
                            title = "WITHOUT GAMMA -- " + title
                        ax[j,i].set_title(title)
                        i += 1
            j += 1
plt.savefig("ALL-SETTINGS-3.pdf")